## MiniCaml con Insiemi

Vogliamo estendere l'interprete visto a lezione, implementando un costrutto che permetta di formare **insiemi di elementi dello stesso tipo**. Esempi di espressioni in questo linguaggio esteso potrebbero essere:
```
{int: 1, 2, 3, 4};;

let x = {int: 1, 2, 3, 4} in 
    insert(6, x);;

let x = {string: "hello", "world", "from", "MiniCaml" } in
    if ("hi" in x) then 0 else 1

let x = {string: "hello", "world", "from", "MiniCaml" } in
let y = {string: "hi" "world"} in
    intersection(x, y)
```

### Es 1: Albero della sintassi astratta

Estendere l'AST di MiniCaml con un **nuovo tipo** e le seguenti **nuove espressioni**:
1. Espressioni per **costruire insiemi**: 
    - una per l'insieme vuoto (di un certo tipo) 
    - una per l'insieme singoletto contenente un dato elemento (di un certo tipo) 
    - e una per l'insieme contenente una data sequenza di elementi distinti (tutti di un certo tipo)
    
    __NOTA:__ Ogni insieme, anche se vuoto, mantiene sempre l'informazione sul tipo di elementi che può contenere. Quindi le espressioni `{int:}` e `{string:}` dovranno essere di tipi diversi...


2. Un **operatore unario** su insiemi:
    - `isEmpty`, che restituisca true solo se l'insieme è vuoto


3. **Operatori fra insiemi ed elementi**:
    - `Insert`, tale che `insert x A` resituisca $A \cup \{x\}$
    - `Remove`, tale che `remove x A` restituisca $A \setminus \{x\}$
    - `Member`, tale che `member x A` restituisca `true` sse $x \in A$


4. **Operatori binari** su insiemi:
    - `Union`, tale che `union A B` restituisca $A \cup B$
    - `Intersection`, tale che `intersection A B` restituisca $A \cap B$
    - `Subset`, tale che `subset A B` restituisca `true` sse $A \subset B$

In [1]:
(* Identificatori *)
type ide = string;;

(*############# tsub sono esattamente i tipi di dato ammissibili per un insieme: interi, stringhe e booleani *)
type tsub = TInt | TBool | TString

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound 
    |TSet of tsub (*#############*)

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (*###### Costruttori di insiemi *)
    | Empty of tsub (* Insieme vuoto *)
    | Singleton of tsub*exp (* Insieme con un solo elemento *)
    | Of of tsub*exp list (* Insieme con uno o più elementi *)
    (*##### Operatori unari su insiemi*)
    | IsEmpty of exp 
    | GetMin of exp 
    | GetMax of exp
    (*##### Operatori binari su insiemi *)
    | Union of exp*exp
    | Intersection of exp*exp 
    | Difference of exp*exp 
    | Insert of exp*exp
    | Remove of exp*exp
    | Contains of exp*exp
    | IsSubset of exp*exp

type ide = string


type tsub = TInt | TBool | TString


type tname =
    TInt
  | TBool
  | TString
  | TClosure
  | TRecClosure
  | TUnBound
  | TSet of tsub


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | Empty of tsub
  | Singleton of tsub * exp
  | Of of tsub * exp list
  | IsEmpty of exp
  | GetMin of exp
  | GetMax of exp
  | Union of exp * exp
  | Intersection of exp * exp
  | Difference of exp * exp
  | Insert of exp * exp
  | Remove of exp * exp
  | Contains of exp * exp
  | IsSubset of exp * exp


### Es 2: Tipi esprimibili

Estendere i tipi esprimibili e la funzione `getType` per comprendere anche i valori insieme.

In [2]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | Set of tsub*(evT list)(*###############*)
    | UnBound

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | Set(t,l) -> TSet(t)(*#########*)
    | UnBound -> TUnBound

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | Set of tsub * evT list
  | UnBound


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


val getType : evT -> tname = <fun>


### Es 3: Typechecking

Estendere la funzione di typechecking in maniera tale che `typecheck(TSet, v)` restituisca `true` se e solo se `v` è un valore di tipo `TSet`, cioè è un insieme di valori, e tutti gli elementi contenuti in `v` sono del tipo adeguato.

In [3]:
(* Errori a runtime *)
exception RuntimeError of string

(* Funzione di upcasting da tsub a tname #############*)
let (upcast : tsub -> tname) = function x -> match x with
    | TInt -> TInt
    | TBool -> TBool
    | TString -> TString

(* Funzione di downcasting da tname a tsub ###########*)
let (downcast : tname -> tsub) = function x -> match x with
    | TInt -> TInt
    | TBool -> TBool
    | TString -> TString
    | _ -> raise ( RuntimeError "Cannot downcast")


let rec allSameType (typ : tname) (l : evT list) = 
    match l with
    | [] -> true
    | h::tl ->  (getType h = typ) && allSameType typ tl


(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    |TSet(subT) -> (*#########*)
            (match y with
                | Set(t, l) -> 
                    if (subT = t) then 
                        allSameType (upcast t) l 
                    else false
                | _ -> false
                )
    |TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )

exception RuntimeError of string


val upcast : tsub -> tname = <fun>


val downcast : tname -> tsub = <fun>


val allSameType : tname -> evT list -> bool = <fun>


val typecheck : tname * evT -> bool = <fun>
